# 3D Mesh Analysis
This notebook contains methods for 3D mesh analysis and visualization. It is dependent on the __plotly__ package for data and mesh visualization, and on common scipy packages like __numpy__ for data processing. All geometric features are computed natively without other external dependencies to ensure correctness.


In [1]:
#Setup and check required packages
import plotly
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
print("Plotly: ",plotly.__version__)
import numpy as np
print("NumPy: ",np.__version__)
import matplotlib as mplt
import matplotlib.pyplot as plt
print("Matplotlib: ", mplt.__version__)

import time

Plotly:  3.10.0
NumPy:  1.16.3
Matplotlib:  3.0.3


https://plot.ly/python/

## Action Items:
* ~~Mesh extraction~~
* ~~Mesh visualization~~
* ~~Normal vector calculation, visualization~~
* ~~Mean curvature calculation, visualization~~
* ~~Corrected curvature calculation, zero handling~~
* ~~Directional curvature components~~
* ~~Normal vector histogram~~
* ~~Directional curvature histograms~~
* ~~Ear tip and nose geometric identification~~
* ~~Improved geometric identification robustness~~
* ~~Region selection based on geometric landmarks~~
* ~~Region selection based on normal vector~~
* ~~Bulk processing~~
* Old metrics versus improved metrics
* ~~Metric analysis between groups~~
* Improve by preprocessing head alignment and position

* ~~Object-oriented~~
* ~~Surface area metrics~~
* ~~Debug landmark creation~~
* ~~Results and group results~~

* ~~Split patient load from region/result update~~
* ~~Multiple regions - front half vs back half of cranium~~
* ~~Result - per-vertex ratio of x to total curvature~~
* Debug directional curvature magnitude
* T test
* Analysis - separation of pre vs post for each metric - T test all
* Analysis of best metric between groups
* Plot metrics versus post-op month to see age correlation

Extracted data features:

    vertices
    faces
    minX/maxX/rangeX
    maxrange
    vertexNeighbors
    faceNeighbors
    faceAreas
    faceNormals
    vertexNormals
    vertexElevation
    vertexAzimuth
    vertexCurvatures
    vertexDirectionalCurvatures
    leftEarIndex, rightEarIndex, nasionIndex, noseIndex

In [2]:
%run Multi_Mesh_Definitions.ipynb

Plotly:  3.10.0
NumPy:  1.16.3
Matplotlib:  3.0.3


Done loading Multi_Mesh_Definitions


BEGIN  - Main Code

In [3]:
#Definitions for multithreaded mesh loading

import os
import re
import multiprocessing
from multiprocessing import Pool

class PatientData ():
    def __init__(self, patientIndex,preOpFile,postOpFile,group,patientNumber,operationType,postOpYears):
        self.patientIndex = patientIndex
        self.preOpFile = preOpFile
        self.postOpFile = postOpFile
        self.group = group
        self.patientNumber = patientNumber
        self.operationType = operationType
        self.postOpYears = postOpYears
        
def createPatientParallel(patientData):
    #print("Starting proc " + str(patientData.patientIndex))
    preOpObjFile = open(patientData.preOpFile,"rt")
    postOpObjFile = open(patientData.postOpFile,"rt")
    preOpMesh = obj_data_to_Mesh(preOpObjFile.read())
    postOpMesh = obj_data_to_Mesh(postOpObjFile.read())
    preOpObjFile.close()
    postOpObjFile.close()
    currPatient=Patient(preOpMesh,postOpMesh,patientData.group,patientData.patientNumber,patientData.operationType,patientData.postOpYears)
    #print("Completing proc " + str(patientData.patientIndex))
    return currPatient



In [4]:
#Add patients to list - multithreaded version

startingDir = '/home/dstow/research/med/scans/SickKids'
dirlist = os.listdir(startingDir)
patientList = list()
patientIndex=0
patientDataList = list()

for group in dirlist:
    groupDir = startingDir+'/'+group
    patientDirList = os.listdir(groupDir)
    for patient in patientDirList:
        patientDir = groupDir+'/'+patient
        preOpObjFile=""
        postOpObjFile=""
        patientNumber=""
        operationType=""
        postOpYears=999
        match=re.search("(\d+)_(\D+)$",patient)
        if(match):
            patientNumber=match.group(1)
            operationType=match.group(2)            
        else:
            print("Error: patient number or operation type not found for "+patient)
            break
        opDirList = os.listdir(patientDir)
        for op in opDirList:
            opDir = patientDir+'/'+op
            if(re.search("^Pre",op)):
                fileDirList = os.listdir(opDir)
                simpFound=False
                for file in fileDirList:
                    if(re.search("simp.obj",file)):
                        simpFound=True
                        preOpFile = opDir+'/'+file
                if(not simpFound):
                    print("Error: *simp.obj file not found for directory "+opDir)
            match=re.search("^Post.*(\d+\.\d+)",op)
            if(match):
                postOpYears=match.group(1)
                fileDirList = os.listdir(opDir)
                simpFound=False
                for file in fileDirList:
                    if(re.search("simp.obj",file)):
                        simpFound=True
                        postOpFile = opDir+'/'+file
                if(not simpFound):
                    print("Error: *simp.obj file not found for directory "+opDir)
        print("Creating meshes for patient file "+patientDir)
        patientDataList.append(PatientData(patientIndex,preOpFile,postOpFile,group,patientNumber,operationType,postOpYears))
        patientIndex=patientIndex+1
        
        
#print( "Calculating geometric results for " + str(patientIndex) + " patients." )
print( "Calculating geometric results for " + str(len(patientDataList)) + " patients..." )
    

#Launch procs
start_time = time.time()
pool = multiprocessing.Pool(processes=64)
patientList = pool.map(createPatientParallel, patientDataList)

    
end_time = time.time()
total_time = end_time - start_time
print("Done! patientList count:",len(patientList)," time:",str(total_time))

Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps2000_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1737_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1849_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1813_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1831_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1804_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1766_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phillips/Ps1760_TCVR
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phillips/Ps1388_ESC
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phill

In [5]:
# Build resultListPre and resultListPost - ordered lists with pre and post op meshes
# These lists are used for later analysis

resultListPre = list()
resultListPost = list()
for currPatient in patientList:
    currResultPre = Results(currPatient,isPre=True)
    currResultPost = Results(currPatient,isPre=False)
    resultListPre.append(currResultPre)
    resultListPost.append(currResultPost)
    
print("resultListPre count: ",len(resultListPre))
print("resultListPost count: ",len(resultListPost))

resultListPre count:  44
resultListPost count:  44


In [6]:
publicHost = True
if(not publicHost):
    #for currPatient in patientList:
    #    plot_mesh_landmarks(currPatient.preOpMesh)
    for i in range(6):
        currPatient = patientList[i]
        currResult = Results(currPatient,isPre=False)
        currPostMesh = currPatient.postOpMesh
        frontVertices = currResult.frontRegionVertices()
        backVertices = currResult.backRegionVertices()
        customVertexColoring=np.ones_like(currPatient.postOpMesh.vertices)
        customVertexColoring[frontVertices,:]=[0.0,1.0,1.0]
        customVertexColoring[backVertices,:]=[1.0,1.0,0.0]
        plot_mesh_custom(currPatient.postOpMesh,customVertexColoring)

In [7]:
#First, sort pre and post lists into lists for each operation type
#Next, build GroupResults and DeltaResults to extract group trends
#Finally, perform statistical testing with GroupCompare for each metric
# Only display the significant metrics


endoResultsPre=list()
escResultsPre=list()
tcvrResultsPre=list()
endoResultsPost=list()
escResultsPost=list()
tcvrResultsPost=list()

#print(len(resultListPre))

significance = 0.01


for currResultPre in resultListPre:
    if(currResultPre.operationType=="Endoscopic"):
        endoResultsPre.append(currResultPre)
    elif(currResultPre.operationType=="ESC"):
        escResultsPre.append(currResultPre)
    elif(currResultPre.operationType=="TCVR"):
        tcvrResultsPre.append(currResultPre)
    else:
        print("ERROR: no valid operation type for patient ")

endoGroupResultsPre=GroupResults(endoResultsPre)
escGroupResultsPre=GroupResults(escResultsPre)
tcvrGroupResultsPre=GroupResults(tcvrResultsPre)

print("Endoscopic Patient Count: ",endoGroupResultsPre.resultCount)
print("ESC Patient Count: ",escGroupResultsPre.resultCount)
print("TCVR Patient Count: ",tcvrGroupResultsPre.resultCount)

#print(len(resultListPost))

for currResultPost in resultListPost:
    if(currResultPost.operationType=="Endoscopic"):
        endoResultsPost.append(currResultPost)
    elif(currResultPost.operationType=="ESC"):
        escResultsPost.append(currResultPost)
    elif(currResultPost.operationType=="TCVR"):
        tcvrResultsPost.append(currResultPost)
    else:
        print("ERROR: no valid operation type for patient ")

endoGroupResultsPost=GroupResults(endoResultsPost)
escGroupResultsPost=GroupResults(escResultsPost)
tcvrGroupResultsPost=GroupResults(tcvrResultsPost)

endoDeltaResults = DeltaResults(endoResultsPre,endoResultsPost)
escDeltaResults = DeltaResults(escResultsPre,escResultsPost)
tcvrDeltaResults = DeltaResults(tcvrResultsPre,tcvrResultsPost)



print("\nInter-Group Distribution Differences - Pre-Op")

print("Endo-ESC")
for currMetric in endoGroupResultsPost.metrics:
    gcPre=GroupCompare(endoGroupResultsPre,escGroupResultsPre,currMetric)
    if(gcPre.p<significance):
        print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)
    
print("Endo-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gcPre=GroupCompare(endoGroupResultsPre,tcvrGroupResultsPre,currMetric)
    if(gcPre.p<significance):
        print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)
   
    
print("ESC-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gcPre=GroupCompare(escGroupResultsPre,tcvrGroupResultsPre,currMetric)
    if(gcPre.p<significance):
        print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)

        
        
print("\nInter-Group Distribution Differences - Post-Op")

print("Endo-ESC")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(endoGroupResultsPost,escGroupResultsPost,currMetric)
    if(gc.p<significance):
        print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)
    
print("Endo-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(endoGroupResultsPost,tcvrGroupResultsPost,currMetric)
    if(gc.p<significance):
        print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)  
    
print("ESC-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(escGroupResultsPost,tcvrGroupResultsPost,currMetric)
    if(gc.p<significance):
        print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)


print("\nInter-Group Distribution Differences - Delta")

print("Endo-ESC")
for currMetric in endoGroupResultsPost.metrics:
    gcDelta=GroupCompare(endoDeltaResults,escDeltaResults,currMetric)
    if(gcDelta.p<significance):
        print("Delta Metric:",gc.metric," p:",gc.p)
    
print("Endo-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gcDelta=GroupCompare(endoDeltaResults,tcvrDeltaResults,currMetric)
    if(gcDelta.p<significance):
        print("Delta Metric:",gc.metric," p:",gc.p)   
    
print("ESC-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gcDelta=GroupCompare(escDeltaResults,tcvrDeltaResults,currMetric)
    if(gcDelta.p<significance):
        print("Delta Metric:",gc.metric," p:",gc.p)
        
fig=go.Figure()
fig.add_trace(go.Scatter(x=endoGroupResultsPre.results["curvYPercentBack"],y=endoGroupResultsPre.results["curvZPercentBack"],mode='markers',name='Endo'))
fig.add_trace(go.Scatter(x=escGroupResultsPre.results["curvYPercentBack"],y=escGroupResultsPre.results["curvZPercentBack"],mode='markers',name='ESC'))
fig.add_trace(go.Scatter(x=tcvrGroupResultsPre.results["curvYPercentBack"],y=tcvrGroupResultsPre.results["curvZPercentBack"],mode='markers',name='TCVR'))
fig.show()



Endoscopic Patient Count:  7
ESC Patient Count:  24
TCVR Patient Count:  13

Inter-Group Distribution Differences - Pre-Op
Endo-ESC
Pre Metric: curvYPercentBack  p: 0.00014448785113447315  change: -0.019103718098090805
Endo-TCVR
Pre Metric: curvYPercentBack  p: 0.005508174677059034  change: -0.01932819670021435
ESC-TCVR
Pre Metric: curvZPercentFront  p: 0.0008179246931474349  change: 0.024653329824177617

Inter-Group Distribution Differences - Post-Op
Endo-ESC
Endo-TCVR
ESC-TCVR

Inter-Group Distribution Differences - Delta
Endo-ESC
Endo-TCVR
ESC-TCVR


In [8]:

# Display all metrics, not just significant metrics



print("\nInter-Group Distribution Differences - Pre-Op")

print("Endo-ESC")
for currMetric in endoGroupResultsPost.metrics:
    gcPre=GroupCompare(endoGroupResultsPre,escGroupResultsPre,currMetric)
    print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)
    
print("Endo-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gcPre=GroupCompare(endoGroupResultsPre,tcvrGroupResultsPre,currMetric)
    print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)
   
    
print("ESC-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gcPre=GroupCompare(escGroupResultsPre,tcvrGroupResultsPre,currMetric)
    print("Pre Metric:",gcPre.metric," p:",gcPre.p, " change:",gcPre.x2-gcPre.x1)

        
        
print("\nInter-Group Distribution Differences - Post-Op")

print("Endo-ESC")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(endoGroupResultsPost,escGroupResultsPost,currMetric)
    print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)
    
print("Endo-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(endoGroupResultsPost,tcvrGroupResultsPost,currMetric)
    print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)  
    
print("ESC-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(escGroupResultsPost,tcvrGroupResultsPost,currMetric)
    print("Post Metric:",gc.metric," p:",gc.p, " change:",gc.x2-gc.x1)


print("\nInter-Group Distribution Differences - Delta")

print("Endo-ESC")
for currMetric in endoGroupResultsPost.metrics:
    gcDelta=GroupCompare(endoDeltaResults,escDeltaResults,currMetric)
    print("Delta Metric:",gc.metric," p:",gc.p)
    
print("Endo-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gcDelta=GroupCompare(endoDeltaResults,tcvrDeltaResults,currMetric)
    print("Delta Metric:",gc.metric," p:",gc.p)   
    
print("ESC-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gcDelta=GroupCompare(escDeltaResults,tcvrDeltaResults,currMetric)
    print("Delta Metric:",gc.metric," p:",gc.p)




Inter-Group Distribution Differences - Pre-Op
Endo-ESC
Pre Metric: curvFront  p: 0.3485160378993908  change: -0.0027720029351769454
Pre Metric: curvXFront  p: 0.27328843165710764  change: -0.0011897996643358121
Pre Metric: curvYFront  p: 0.4532276839429771  change: -0.0009259903577497428
Pre Metric: curvZFront  p: 0.3194023335210436  change: -0.0012926867207060412
Pre Metric: curvXPercentFront  p: 0.9521163973426393  change: -0.000508600701194184
Pre Metric: curvYPercentFront  p: 0.11584106819377381  change: 0.01122740607944106
Pre Metric: curvZPercentFront  p: 0.11663896986451508  change: -0.012699053990331333
Pre Metric: curvBack  p: 0.018648325791579483  change: -0.008014701819125381
Pre Metric: curvXBack  p: 0.021207130983966385  change: -0.0027120113126371213
Pre Metric: curvYBack  p: 0.024687427407372703  change: -0.003744997811213093
Pre Metric: curvZBack  p: 0.011300142354394489  change: -0.003312376256137877
Pre Metric: curvXPercentBack  p: 0.46236271806853313  change: 0.0036

In [9]:
#print(len(resultListPre))
print(len(resultListPost))

significance = 0.05

groupResultsPre=GroupResults(resultListPre)
groupResultsPost=GroupResults(resultListPost)
deltaResults=DeltaResults(resultListPre,resultListPost)

for currMetric in groupResultsPre.metrics:
    gc=GroupCompare(groupResultsPre,groupResultsPost,currMetric)
    if(gc.p<significance):
        print("Metric:",gc.metric," p:",gc.p)
    #print(gc.metric,",",gc.p)
    
#fig=go.Figure()
#fig.add_trace(go.Scatter(x=groupResultsPre.results["curvYPercentBack"],y=groupResultsPre.results["curvYPercentBack"],mode='markers',name='Pre'))
#fig.add_trace(go.Scatter(x=groupResultsPost.results["curvYPercentBack"],y=groupResultsPost.results["curvYPercentBack"],mode='markers',name='Post'))
#fig.show()

#plot_hist(groupResultsPre.results["curvYPercentBack"],"Pre-curvYPercentBack")
#plot_hist(groupResultsPost.results["curvYPercentBack"],"Post-curvYPercentBack")
#plot_hist_double(groupResultsPre.results["curvYPercentBack"],"Pre-curvYPercentBack",groupResultsPost.results["curvYPercentBack"],"Post-curvYPercentBack")

#for currMetric in deltaResults.metrics:
#    print(currMetric,deltaResults.means[currMetric],deltaResults.stds[currMetric])

44
Metric: curvXPercentFront  p: 0.01612537410262456
Metric: curvZPercentFront  p: 0.005082335223090565
Metric: curvBack  p: 1.0437173747435959e-05
Metric: curvXBack  p: 0.00017322933455435883
Metric: curvYBack  p: 1.5595986049025408e-06
Metric: curvZBack  p: 1.5631764372082986e-05
Metric: curvXPercentBack  p: 1.7562557969442197e-05
Metric: curvYPercentBack  p: 7.855914100931243e-09
Metric: curvZPercentBack  p: 0.002595740343902082


Last Updated: 1/17/20